# Spark SQL

First, let's create a Spark session.

In [1]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Spark course - Spark SQL").\
    master("local[*]").enableHiveSupport().getOrCreate()

sc = spark.sparkContext

File `/home/spark/first-edition/ch05/italian-posts.csv` contains anonymized data dump of questions and
answers from the Stack Exchange Italian language community as tilda-separated values. Each line contains these fields:   
* commentCount - Number of comments related to the question/answer
* lastActivityDate - Date and time of the last modification
* ownerUserId - User ID of the owner
* body - Textual contents of the question/answer
* score - Total score based on upvotes and downvotes
* creationDate - Date and time of creation
* viewCount - View count
* title - Title of the question
* tags - Set of tags the question has been marked with 
* answerCount - Number of related answers
* acceptedAnswerId - If a question contains the ID of its accepted answer
* postTypeId - Type of the post; 1 is for questions, 2 for answers
* id - Post’s unique ID
   
Load the file into `itPostsRows` RDD.

In [2]:
itPostsRows = sc.textFile("/home/spark/first-edition/ch05/italianPosts.csv")

Create a Spark schema (a `StructType` object) with the following columns:

| Column name | Type | Nullable |
| ----------- | ---- | -------- |
| commentCount | integer | true |
| lastActivityDate | timestamp | true |
| ownerUserId | long | true |
| body | string | true |
| score | integer | true |
| creationDate | timestamp | true |
| viewCount | integer | true |
| title | string | true |
| tags | string | true |
| answerCount | integer | true |
| acceptedAnswerId | long | true |
| postTypeId | long | true |
| id | long | false |


In [3]:
from pyspark.sql.types import *
postSchema = StructType([
  StructField("commentCount", IntegerType(), True), 
  StructField("lastActivityDate", TimestampType(), True), 
  StructField("ownerUserId", LongType(), True),
  StructField("body", StringType(), True),
  StructField("score", IntegerType(), True),
  StructField("creationDate", TimestampType(), True),
  StructField("viewCount", IntegerType(), True),
  StructField("title", StringType(), True),
  StructField("tags", StringType(), True),
  StructField("answerCount", IntegerType(), True),
  StructField("acceptedAnswerId", LongType(), True),
  StructField("postTypeId", LongType(), True),
  StructField("id", LongType(), False)]
)

The following code will make parsing easier:

In [4]:
from pyspark.sql import Row
import datetime

def toint(s):
    try:
        return int(s)
    except:
        return None
    
def tots(s):
    try:
        return datetime.datetime.strptime(s, "%Y-%m-%d %H:%M:%S.%f")
    except:
        return None

def stringToRow(row):
    r = row.split("~")
    return Row(toint(r[0]),
               tots(r[1]),
               toint(r[2]),
               r[3],
               toint(r[4]),
               tots(r[5]),
               toint(r[6]),
               r[7],
               r[8],
               toint(r[9]),
               toint(r[10]),
               toint(r[11]),
               int(r[12]))

Now use the above methods to create an RDD of Rows and then a DataFrame with the above schema. Call it `postsDf`.

In [5]:
rowRDD = itPostsRows.map(lambda row: stringToRow(row))
postsDf = spark.createDataFrame(rowRDD, postSchema)

Print schema of the `postsDf` DataFrame.

In [6]:
postsDf.printSchema()

root
 |-- commentCount: integer (nullable = true)
 |-- lastActivityDate: timestamp (nullable = true)
 |-- ownerUserId: long (nullable = true)
 |-- body: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- creationDate: timestamp (nullable = true)
 |-- viewCount: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- answerCount: integer (nullable = true)
 |-- acceptedAnswerId: long (nullable = true)
 |-- postTypeId: long (nullable = true)
 |-- id: long (nullable = false)



Print out the first 20 rows from the DataFrame.

In [7]:
postsDf.show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|           4|2013-11-11 18:21:...|         17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                    |                    |       null|            null|         2|1165|
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|          1|            null|         1|1166|
|           2|2013-11-10 20:31:...|

Print out the first 5 rows.

In [13]:
postsDf.show(5)

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+
|           4|2013-11-11 18:21:...|         17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                    |                    |       null|            null|         2|1165|
|           5|2013-11-10 20:31:...|         12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|          1|            null|         1|1166|
|           2|2013-11-10 20:31:...|

Get the columns from `postsDf`.

In [15]:
postsDf.columns

['commentCount',
 'lastActivityDate',
 'ownerUserId',
 'body',
 'score',
 'creationDate',
 'viewCount',
 'title',
 'tags',
 'answerCount',
 'acceptedAnswerId',
 'postTypeId',
 'id']

Print out 20 rows of `id` and `body` columns from `postsDf`.

In [16]:
postsDf.select(postsDf.id, postsDf.body).show()

+----+--------------------+
|  id|                body|
+----+--------------------+
|1165|&lt;p&gt;The infi...|
|1166|&lt;p&gt;Come cre...|
|1167|&lt;p&gt;Il verbo...|
|1168|&lt;p&gt;As part ...|
|1169|&lt;p&gt;&lt;em&g...|
|1170|&lt;p&gt;There's ...|
|1171|&lt;p&gt;As other...|
|1172|&lt;p&gt;The expr...|
|1173|&lt;p&gt;When I w...|
|1174|&lt;p&gt;Wow, wha...|
|1175|&lt;p&gt;Suppose ...|
|1176|&lt;p&gt;Except w...|
|1177|&lt;p&gt;Both you...|
|1178|&lt;blockquote&gt...|
|1179|&lt;p&gt;Comparin...|
|1180|&lt;p&gt;Using th...|
|1181|&lt;p&gt;I would ...|
|1182|&lt;p&gt;Putting ...|
|1183|&lt;p&gt;Many peo...|
|1184|&lt;p&gt;Sono un'...|
+----+--------------------+
only showing top 20 rows



As you can see, the body column contains escaped HTML characters. We can make that more readable by applying Python's `html.unescape` method. Write a UDF and use it to unescape text from the body column. Preserve the name of the column (you will need to use a temporary name, `drop` the old column, then rename the new one).

In [8]:
import html
unescape = spark.udf.register('unescape', lambda s: html.unescape(s))
postsDf = postsDf.withColumn('bodynew', unescape(postsDf.body)).drop("body").withColumnRenamed('bodynew', 'body')

How many posts contain the word "Italiano"?

In [9]:
postsDf.where(postsDf.body.contains("Italiano")).count()

46

How many questions don't have an accepted answer? (Hint: `postTypeId` is 1 and `acceptedAnswerId` is null)

In [10]:
postsDf.filter((postsDf.postTypeId == 1) & (postsDf.acceptedAnswerId.isNull())).count()

152

Get the first 10 bodies of such questions. (Hint: use `limit`)

In [11]:
import html
[html.unescape(r.body) for r in postsDf.filter((postsDf.postTypeId == 1) & (postsDf.acceptedAnswerId.isNull())).limit(10).select("body").collect()]

['<p>Come credo sia conosciuto da tutti quelli che usano viaggiare con l\'automobile, molti italiani hanno uno strano rapporto con gli abbaglianti; alcuni li amano così tanto che preferiscono mantenerli sempre accesi, altri invece li usano per segnalare, se non addirittura per comunicare informazioni di vario genere, dalla presenza di autovelox alla protesta per presunte violazioni del codice della strada.</p><p>Al di lá delle considerazioni e dei commenti circa queste abitudini, mi piacerebbe sapere se il verbo "sfanagliare" è normalmente usato, e compreso, in tutte le regioni italiane o se, magari, ci sono altri verbi in uso, purchè simpatici come quello.</p><p>Laddove qualcuno non avesse compreso l\'uso del aforementioned verbo, ecco un esempio:</p><blockquote>  <p>"Ehi!" - dice il marito a sua moglie - "Quello li mi sta sfanagliando, st***o!"</p>    <p>E la moglie, "Caro, rallenta; magari più avanti c\'è un autovelox, cribbio!"</p></blockquote>',
 '<p>Quando soggiorno in Emilia-Rom

Calculate "views per score ratio" (number of views needed to increase score by one) of questions (postTypeId is 1) as column "vsratio" and show posts whose "vsratio" is less than 5.

In [12]:
postsDf.filter(postsDf.postTypeId == 1).\
  withColumn("ratio", postsDf.viewCount / postsDf.score).\
  where("ratio < 5").show()

+------------+--------------------+-----------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+--------------------+-------------------+
|commentCount|    lastActivityDate|ownerUserId|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|                body|              ratio|
+------------+--------------------+-----------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+--------------------+-------------------+
|           1|2013-11-12 12:53:...|         99|   -3|2013-11-12 10:57:...|       68|Past participle o...|&lt;grammar&gt;&l...|          3|            1216|         1|1203|<p>I can't figure...|-22.666666666666668|
|           6|2013-11-26 19:12:...|         12|   -3|2013-11-21 21:12:...|       79|È &quot;memorioso...|  &lt;word-usage&gt;|          2|          

Find the body of the question that was active for the largest amount of time. (Use the `datediff` function to find the difference between `lastActivityDate` and `creationDate`.) First import pyspark functions as follows:

In [13]:
import pyspark.sql.functions as F

In [14]:
ddiffdf = postsDf.filter(postsDf.postTypeId == 1).\
  withColumn("activePeriod", F.datediff(postsDf.lastActivityDate, postsDf.creationDate))
ddiffdf.orderBy(ddiffdf.activePeriod.desc()).head().body

'<p>The plural of <em>braccio</em> is <em>braccia</em>, and the plural of <em>avambraccio</em> is <em>avambracci</em>.</p><p>Why are the plural of those words so different, if they both are referring to parts of the human body, and <em>avambraccio</em> derives from <em>braccio</em>?</p>'

Find the minimum, maximum and average score of all *questions* and the count of all questions. Give those columns names "Minimum score", "Maximum score", "Average score" and "Total count", respectively. (Hint: use `alias` for renaming columns.)

In [15]:
postsDf.filter(postsDf.postTypeId == 1).select(F.min(postsDf.score).alias("Minimum score"), 
  F.max(postsDf.score).alias("Maximum score"),
  F.avg(postsDf.score).alias("Average score"), 
  F.count(postsDf.score).alias("Total count")).show()

+-------------+-------------+-----------------+-----------+
|Minimum score|Maximum score|    Average score|Total count|
+-------------+-------------+-----------------+-----------+
|           -3|           20|4.735294117647059|        374|
+-------------+-------------+-----------------+-----------+



Find the number of posts per ownerUserId, tags and postTypeId (use groupBy) and find 10 rows with largest counts.

In [16]:
postsDf.groupBy(postsDf.ownerUserId, postsDf.tags, postsDf.postTypeId).\
  count().orderBy(F.col('count').desc()).show(10)

+-----------+----+----------+-----+
|ownerUserId|tags|postTypeId|count|
+-----------+----+----------+-----+
|        193|    |         2|   74|
|         37|    |         2|   59|
|         70|    |         2|   52|
|          8|    |         4|   52|
|          8|    |         5|   52|
|          8|    |         2|   36|
|         95|    |         2|   29|
|         17|    |         2|   28|
|         63|    |         2|   28|
|         18|    |         2|   27|
+-----------+----+----------+-----+
only showing top 10 rows



Find the last activity date and the maximum post score per user.

In [44]:
postsDf.groupBy(postsDf.ownerUserId).\
  agg(F.max(postsDf.lastActivityDate), F.max(postsDf.score)).show(10)

+-----------+---------------------+----------+
|ownerUserId|max(lastActivityDate)|max(score)|
+-----------+---------------------+----------+
|        270| 2014-02-25 17:43:...|         1|
|        730| 2014-07-12 00:58:...|        12|
|        720| 2014-07-07 21:33:...|         1|
|         19| 2013-11-27 14:21:...|        10|
|        348| 2014-01-06 13:00:...|         5|
|        415| 2014-08-25 00:23:...|         5|
|        656| 2014-05-27 19:30:...|         9|
|        736| 2014-07-15 11:09:...|         1|
|         22| 2014-09-10 07:15:...|        19|
|        198| 2013-12-18 15:57:...|         5|
+-----------+---------------------+----------+
only showing top 10 rows



Now for each user and activity find the time elapsed since their last activity in minutes (use `pyspark.sql.window.Window` and `lag`).

In [27]:
from pyspark.sql.window import Window
postsDf.select('ownerUserId', 'lastActivityDate', 
    ((postsDf.lastActivityDate.cast('long') - 
          F.lag(postsDf.lastActivityDate).over(Window.partitionBy('ownerUserId').orderBy('lastActivityDate')).cast('long')
      )/60).cast('int').alias('inactivePeriod')).\
  show(20, False)

+-----------+-----------------------+--------------+
|ownerUserId|lastActivityDate       |inactivePeriod|
+-----------+-----------------------+--------------+
|270        |2014-02-25 17:16:13.043|null          |
|270        |2014-02-25 17:43:08.443|26            |
|730        |2014-07-12 00:58:12.843|null          |
|720        |2014-07-07 21:33:30.99 |null          |
|19         |2013-11-12 11:24:03.19 |null          |
|19         |2013-11-12 11:24:33.87 |0             |
|19         |2013-11-12 11:48:52.947|24            |
|19         |2013-11-12 11:57:44.983|8             |
|19         |2013-11-12 12:14:42.707|16            |
|19         |2013-11-12 16:38:23.007|263           |
|19         |2013-11-12 17:42:52.417|64            |
|19         |2013-11-13 00:32:20.64 |409           |
|19         |2013-11-13 18:45:13.39 |1092          |
|19         |2013-11-13 18:50:38.11 |5             |
|19         |2013-11-13 19:16:25.333|25            |
|19         |2013-11-19 18:01:28.993|8565     

Load `/home/spark/first-edition/ch05/italianVotes.csv` file into an RDD called `itVotesRaw` and split each line by tilda sign ("~").

In [45]:
itVotesRaw = sc.textFile("/home/spark/first-edition/ch05/italianVotes.csv").\
  map(lambda x: x.split("~"))

Execute the following cell to create the `votesDf` DataFrame.

In [49]:
itVotesRows = itVotesRaw.map(lambda row: Row(int(row[0]), int(row[1]), 
  int(row[2]), datetime.datetime.strptime(row[3], "%Y-%m-%d %H:%M:%S.%f")))
votesSchema = StructType([
  StructField("id", LongType(), False),
  StructField("postId", LongType(), False),
  StructField("voteTypeId", IntegerType(), False),
  StructField("creationDate", TimestampType(), False)])
votesDf = spark.createDataFrame(itVotesRows, votesSchema)

Look at the contents of `votesDf`.

In [50]:
votesDf.show()

+----+------+----------+-------------------+
|  id|postId|voteTypeId|       creationDate|
+----+------+----------+-------------------+
|2657|   135|         2|2013-11-22 00:00:00|
|2658|   142|         2|2013-11-22 00:00:00|
|2659|   142|         1|2013-11-22 00:00:00|
|2660|   140|         2|2013-11-22 00:00:00|
|2661|   140|         1|2013-11-22 00:00:00|
|2662|  1354|         2|2013-11-22 00:00:00|
|2663|  1356|         2|2013-11-22 00:00:00|
|2664|  1353|         2|2013-11-22 00:00:00|
|2665|  1351|         2|2013-11-22 00:00:00|
|2667|  1357|         2|2013-11-22 00:00:00|
|2668|  1357|         2|2013-11-22 00:00:00|
|2669|  1351|         2|2013-11-22 00:00:00|
|2670|  1351|         1|2013-11-22 00:00:00|
|2671|  1352|         2|2013-11-22 00:00:00|
|2672|  1349|         2|2013-11-22 00:00:00|
|2673|  1357|         2|2013-11-22 00:00:00|
|2674|  1342|         2|2013-11-22 00:00:00|
|2675|  1340|         2|2013-11-22 00:00:00|
|2676|  1321|         2|2013-11-22 00:00:00|
|2677|  13

The file contains votes (id, vote type and vote creation date) and references posts by their ID.    
   
Join `postsDf` and `votesDf` by post ID and show the resulting DataFrame's contents.

In [51]:
postsDf.join(votesDf, postsDf["id"] == votesDf["postId"]).show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+----+------+----------+-------------------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|  id|postId|voteTypeId|       creationDate|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+----+------+----------+-------------------+
|           1|2013-11-06 00:22:...|         18|&lt;p&gt;&quot;De...|    3|2013-11-06 00:05:...|     null|                    |                    |       null|            null|         2|  26|  62|    26|         2|2013-11-06 00:00:00|
|           1|2013-11-06 00:22:...|         18|&lt;p&gt;

Now you have several columns with identical names, which is not good. Repeat the join, but rename columns `id` and `creationDate` of `votesDf` to something unique. (Hint: use `withColumnRenamed`)

In [52]:
postsVotes = postsDf.join(votesDf.withColumnRenamed("id", "voteId").withColumnRenamed("creationDate", "voteCreationDate"), 
  postsDf["id"] == votesDf["postId"])
postsVotes.show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+------+------+----------+-------------------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|               title|                tags|answerCount|acceptedAnswerId|postTypeId|  id|voteId|postId|voteTypeId|   voteCreationDate|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+--------------------+--------------------+-----------+----------------+----------+----+------+------+----------+-------------------+
|           1|2013-11-06 00:22:...|         18|&lt;p&gt;&quot;De...|    3|2013-11-06 00:05:...|     null|                    |                    |       null|            null|         2|  26|    62|    26|         2|2013-11-06 00:00:00|
|           1|2013-11-06 00:22:...|         18|&

Save the resulting table as table "postVotes".

In [54]:
postsVotes.write.saveAsTable("postsVotes")

Execute the following query and show the results:   
`SELECT * FROM POSTSVOTES WHERE ownerUserId = 18`

In [55]:
spark.sql("SELECT * FROM POSTSVOTES WHERE ownerUserId = 18").show()

+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+----+-----------+----------------+----------+----+------+------+----------+-------------------+
|commentCount|    lastActivityDate|ownerUserId|                body|score|        creationDate|viewCount|title|tags|answerCount|acceptedAnswerId|postTypeId|  id|voteId|postId|voteTypeId|   voteCreationDate|
+------------+--------------------+-----------+--------------------+-----+--------------------+---------+-----+----+-----------+----------------+----------+----+------+------+----------+-------------------+
|           5|2014-01-17 10:54:...|         18|&lt;p&gt;The past...|    7|2014-01-17 10:40:...|     null|     |    |       null|            null|         2|1596|  4089|  1596|         2|2014-01-17 00:00:00|
|           5|2014-01-17 10:54:...|         18|&lt;p&gt;The past...|    7|2014-01-17 10:40:...|     null|     |    |       null|            null|         2|1596|  4116|  15

List all the tables in the Spark catalog.

In [ ]:
spark.catalog.listTables()

List all the columns of the 'postsVotes' table using the Spark catalog.

In [ ]:
spark.catalog.listColumns('postsvotes')